# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [7]:
# Let's just make sure the model is loaded
!ollama pull llama3.2

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Automate content creation for social media, blogs, and websites, such as generating product descriptions, blog posts, and even entire articles.
2. **Product Design**: Use generative AI to design new products, packaging, and branding materials, reducing the need for manual design processes.
3. **Marketing Campaigns**: Create personalized marketing campaigns using generative AI-powered tools, such as generating ad copy, images, and videos tailored to individual customers.
4. **Customer Service Chatbots**: Develop conversational AI chatbots that can respond to customer inquiries, providing 24/7 support and improving customer experience.
5. **Data Analytics**: Use generative AI to analyze large datasets, identify patterns, and predict trends, enabling businesses to make data-driven decisions.
6. **Virtual Assistants**: Create virtual assistants that can help with tasks such as

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [10]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and even entire e-books.
2. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized content, such as emails, ads, and landing pages, that are tailored to specific customer segments.
3. **Product Design**: Generative AI can be used to design new products, such as furniture, clothing, or electronics, by generating 2D and 3D designs from user input.
4. **Customer Service**: Generative AI-powered chatbots can help provide 24/7 customer support by generating responses to common queries and providing basic troubleshooting assistance.
5. **Image and Video Generation**: Generative AI can be used to generate high-quality images and videos that can be used in various applications such as advertising, product showcases, 

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [162]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import json
import heapq
import datetime
from dateutil.relativedelta import relativedelta
import itertools

In [53]:
load_dotenv(override=True)
alphavantage_api_key = os.getenv('ALPHAVANTAGE_API_KEY')
marketaux_api_key = os.getenv('MARKETAUX_API_KEY')

In [94]:
def items_until_key(my_dict, target_key_as_date):
    """Returns a new dictionary containing items up to and including the target key."""
    new_dict = {}
    found = False
    for key, value in my_dict.items():
        new_dict[key] = value
        key_as_date = datetime.datetime.strptime(key, "%Y-%m-%d").date()
        if key_as_date < target_key_as_date:
            found = True
            break # Stop iterating once target key is found
    if found:
        print(f'found key with date: {target_key_as_date}, key: {key}')
        return new_dict
    else:
        return my_dict# or raise an exception if you prefer

In [102]:
def get_max_gain_and_fall_days(ticker):
    stockData = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={ticker}&apikey={alphavantage_api_key}')
    weeklyTimeSeries = json.loads(stockData.content)['Weekly Time Series']
    print('firstRow: ' + str(list(weeklyTimeSeries.keys())[0]))

    #TODO - Get date for 3 years before now, consider only 3 years most recent data
    current_date = datetime.date.today()
    three_years_ago = current_date - relativedelta(years=3)

    weeklyTimeSeries = items_until_key(weeklyTimeSeries, three_years_ago)

    dates = list(weeklyTimeSeries.keys())
    close_prices = []
    for date in dates:
        close_prices.append(float(weeklyTimeSeries[date]['4. close']))
    
    top_5_max = []  # Use a min-heap for max
    top_5_min = []  # Use a max-heap for min (by negating values)
    
    for i in range(len(close_prices) - 1):
        diff = close_prices[i+1] - close_prices[i]
        keys = (dates[i], dates[i+1])
    
        heapq.heappush(top_5_max, (diff, keys))
        if len(top_5_max) > 5:
            heapq.heappop(top_5_max)
            
        heapq.heappush(top_5_min, (-diff, keys)) #Negate the value for min heap
        if len(top_5_min) > 5:
            heapq.heappop(top_5_min)
    
    top_5_max.sort(reverse=True)
    top_5_min.sort(reverse=True) #Reverse since we negated
    
    result = {
        "gain": [],
        "fall": []
    }
    
    for diff, keys in top_5_max:
        result["fall"].append({
            "difference": diff,
            "from": keys[1],
            "to": keys[0]
        })
    
    for neg_diff, keys in top_5_min:
        diff = -neg_diff
        result["gain"].append({
            "difference": diff,
            "from": keys[1],
            "to": keys[0]
        })

    json_result = json.dumps(result, indent=4)
    # print(json_result)
    return json_result

In [158]:
diff_days_for_googl = get_max_gain_and_fall_days("GOOG")
print(f'diff_days_for_goog: {diff_days_for_googl}')

firstRow: 2025-01-10
found key with date: 2022-01-13, key: 2022-01-07
diff_days_for_goog: {
    "gain": [
        {
            "difference": -221.75,
            "from": "2022-07-01",
            "to": "2022-07-08"
        },
        {
            "difference": -213.45000000000027,
            "from": "2022-06-17",
            "to": "2022-06-24"
        },
        {
            "difference": -194.5300000000002,
            "from": "2022-01-28",
            "to": "2022-02-04"
        },
        {
            "difference": -126.51999999999998,
            "from": "2022-03-11",
            "to": "2022-03-18"
        },
        {
            "difference": -94.39999999999964,
            "from": "2022-03-18",
            "to": "2022-03-25"
        }
    ],
    "fall": [
        {
            "difference": 2146.98,
            "from": "2022-07-15",
            "to": "2022-07-22"
        },
        {
            "difference": 193.88999999999987,
            "from": "2022-01-14",
            

In [154]:
diff_days_for_aapl = get_max_gain_and_fall_days("AAPL")
print(f'diff_days_for_aapl: {diff_days_for_aapl}')

KeyError: 'Weekly Time Series'

In [170]:
print(f'diff_days_for_nflx: {get_max_gain_and_fall_days("NFLX")}')

KeyError: 'Weekly Time Series'

In [160]:
def get_news_for_ticker_for_date(ticker, start_date, end_date):
    start_date_reformat = start_date.replace('-', '') + 'T0000'
    end_date_reformat = end_date.replace('-', '') + 'T0000'
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&time_from={start_date_reformat}&time_to={end_date_reformat}&apikey={alphavantage_api_key}'
    print(url)
    stories_response = requests.get(url)
    news_data = json.loads(stories_response.content)
    # print(stories_response.content)
    news_title_data = []

    if "feed" in news_data:
        for news_item in news_data["feed"]:
            if "title" in news_item:
                news_title_data.append(news_item["title"])

    return news_title_data

In [120]:
news_titles_googl = get_news_for_ticker_for_date('GOOG', '2022-07-15', '2022-07-22')
print(news_titles_googl)

['U.S. Postal Service Honors Folk Singer Pete Seeger', 'Cancer Research UK and Sosei Heptares sign agreement to advance cancer immunotherapy candidate into clinical trials', 'Cancer Research UK and Sosei Heptares sign agreement to advance cancer immunotherapy candidate into clinical trials', 'This Week In XR: Face Tracking Avatars Score Investment, Google AR Glasses Tests, AR Opera Glasses', 'QCOM Stock, Apple Surge Off Lows Ahead Of Quarterly Earnings Reports', "How Pink Sauce became TikTok's most viral-and controversial-condiment", "Snap's report incinerates $80 billion in ad industry market cap", 'What the first 7 public hearings of the House select committee have revealed about the Jan. 6, 2021, insurrection - a recap', 'Snap To Slow Hiring After Dismal Earnings Pummel Stock Price', 'Business Highlights: Cool housing market, jobless claims', "Why Amazon's three biggest deals ever were for a grocery chain, movie studio, and health care provider", 'Snap stock plunges 25% as advertisi

In [80]:
news_titles_aapl = get_news_for_ticker_for_date('AAPL', '2022-08-28', '2022-09-04')
print(news_titles_aapl)

['RiverPark Advisors LLC Lowers Position in Apple Inc. (NASDAQ:AAPL)', 'Apple Inc. (NASDAQ:AAPL) is Checchi Capital Advisers LLC’s 3rd Largest Position', 'Tredje AP fonden Purchases 41,231 Shares of Apple Inc. (NASDAQ:AAPL)']


In [168]:
def get_news_titles_for_max_gains_and_falls(ticker):
    news_titles = []
    gains_and_falls = json.loads(get_max_gain_and_fall_days(ticker))
    
    for high_fall in gains_and_falls["gain"]:
        news_titles.append(get_news_for_ticker_for_date(ticker, high_fall["from"], high_fall["to"]))

    print(news_titles)

In [171]:
get_news_titles_for_max_gains_and_falls("NFLX")

firstRow: 2025-01-13
found key with date: 2022-01-13, key: 2022-01-07
https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NFLX&time_from=20240119T0000&time_to=20240126T0000&apikey=JF8HN2DRGVTEWGS8
https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NFLX&time_from=20241115T0000&time_to=20241122T0000&apikey=JF8HN2DRGVTEWGS8
https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NFLX&time_from=20221014T0000&time_to=20221021T0000&apikey=JF8HN2DRGVTEWGS8
https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NFLX&time_from=20241129T0000&time_to=20241206T0000&apikey=JF8HN2DRGVTEWGS8
https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=NFLX&time_from=20231013T0000&time_to=20231020T0000&apikey=JF8HN2DRGVTEWGS8
[['News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business', 'News industry off to brutal 2024 start as mass layoffs devastate publishers, raisi

In [150]:
def user_prompt_for(stories):
    user_prompt = f"You are looking at a set of news stories"
    user_prompt += "\nFind 3 common threads linking the below news stories that may have led to steep rise or fall in stocks, the threads don't need to link all stories but should link more than at at least 2, if less than 3 commanality exists then provide those\n\n"
    user_prompt += stories
    return user_prompt

In [156]:
system_prompt = "You are an assistant that analyzes news stories that correspond to steep rises or falls in stocks and identifies common links among those stories. Respond in markdown."

In [166]:
# For Netflix fall - news_titles = [[], ["Here's everything new coming to HBO Max in May 2022 -- and what's leaving", 'Why Netflix Stock Flopped Again Today', 'Disney stock tumbles amid Florida bill controversy', 'Dow falls 368 points as Federal Reserve hints at raising interest rates', "If You Invested $1,000 In Netflix The Last Time The Company Lost Subscribers, Here's How Much You'd Have Now", '3 Reasons to Buy Netflix, and 1 Reason to Sell', 'CNN+ to shut down by end of April due to new management decision', "DraftKings Stock Falls To 52-Week Low: What's Going On?", "For Ackman's Pershing Square, a Record of Big Hits and Misses", 'Markets Fall As Investors Weigh Earnings', 'Dow Jones Falls As Powell Says This On Inflation; Elon Musk Makes Twitter Move; Tesla Stock Gains Fade', 'CNN+ Failed for the Reason Netflix Struggles and Disney+ Thrives', "Real estate ETFs see these types of properties holding up in 'tricky' time for investing", 'Roth IRAs, ESG Stocks, Netflix: Community Conversations', "CNN's streaming service shutting down a month after launch", "These Netflix Analysts Aren't Confident Of A Turnaround Within The Next 18 Months", 'KeyCorp  ( KEY )  Q1 2022 Earnings Call Transcript', 'Boat Rocker Media Announces Details of Q1 2022 Financial Results Conference Call', 'KR: Think the Market is In for a Rough Ride? Then Add These 5 Stocks to Your Portfolio', "Nvidia Stock Is Sliding: What's Going On?", 'Roughly 1 In 10 Adults Use Netflix Password From Another Household-With Baby Boomers The Biggest Culprits-Survey Suggests', 'What Are Whales Doing With Netflix', 'Why Nvidia Stock Fell Hard Today', 'Is AT&T Stock Back in Action and Worth Buying Now?', 'Dow Drops Over 300 Points, Yields Surge After Powell Signals Bigger Rate Hikes Starting In May', 'Carvana Soared but Now Falls Behind Auto-Dealer Rivals', 'HBO Max And HBO Rise To 76.8M Global Subscribers', "Searches for 'Buy Netflix Stock' Explode 931% Following Share Price Plummet For Streaming Giant", "Maybe Netflix's Problem is Its Shows, Not Password Sharing", 'Streaming TV With Ads Puts Netflix And Its Peers At A Crossroads', 'Will DraftKings Inc.  ( DKNG )  Be On Your List Of Penny Stocks In 2022?', 'AT&T;  ( T )  Q1 2022 Earnings Call Transcript', 'Stocks That Hit 52-Week Lows On Thursday', "CNN's streaming service shutting down a month after launch", 'Move Over Netflix? HBO Max Is Gaining Subscribers And Market Share', "CNN's streaming service shutting down a month after launch", "Netflix Delivers Verdict On Streaming's Future: It's TV, Stupid", "10 Stocks Moved By Traders On Thursday's CNBC's 'Fast Money: Halftime Report'", 'Trading Tesla Stock After Impressive Earnings Beat', 'Stocks making the biggest moves midday: United Airlines, AT&T, Tesla and more', 'OnePlus TV 43 Y1S Pro review: Style and substance on a budget | The Financial Express', "Disney Stock Provides Meat For The Bears: Here's Where To Watch For The Bounce", "'Doctor Strange,' 'Thor' And 'Jurassic World' Sequels Top Fandango Summer Movie Poll", 'Better Buy: Netflix vs. Twitter', 'DARK & LOVELY AND STORM REID HOST BUILDING BEAUTIFUL FUTURES MASTERCLASS: "A BLACK GIRL\'S GUIDE TO MENTORSHIP"', 'Transportation: Strong Earnings Form Railroad And Airline Stocks Boost the Sector', 'Netflix Looks to 100 Million Existing Viewers to Boost Revenue', 'U.S. Stocks: Wall Street rises on strong earnings from Tesla, airlines | The Financial Express', 'US stocks rise on upbeat earnings', 'This 26-year-old Google engineer earns $270,000 a year and still uses coupons'], ['Big Q1 Beats After the Close; Markets Struggle to Gain', 'Netflix forces Wall Street to focus on profit and revenue with decision to stop reporting subscriber numbers in 2025', "Netflix keeps adding tons of subscribers from its password-sharing crackdown - and it's good news for Disney", 'Netflix is going to stop telling the world how many subscribers it has. Why?', 'Netflix  ( NFLX )  Beats Q1 Earnings and Revenue Estimates', 'Netflix now has nearly 270 million subscribers after another strong showing to begin 2024', 'Market About To Break Power Trend; Netflix Users Jump', 'Netflix Beats Q1 Targets But Comes Up Short With Q2 Sales Outlook', 'Netflix Q1 Earnings Highlights: Revenue Beat, EPS Beat, 9.33 Million Paid Subs Added, Ad-Tier Plan Update And More - Netflix  ( NASDAQ:NFLX ) ', 'Netflix smashed expectations, adding way more subscribers than Wall Street could have hoped for', 'Trump Media stock rebounds 47% after brutal post-IPO sell-off', "Caitlin Clark salary criticisms a 'false narrative,' says WNBA commissioner Cathy Engelbert", 'Stock market today: Stocks slide for 5th session as Fed officials cast more doubt on rate cuts', "Netflix's 'Baby Reindeer' Is An Incredibly Bizarre, Brilliantly Told True Story", "Britain's Prince Harry formally confirms he is now a US resident", "Netflix Renews And Also Cancels 'The Witcher' As First Glimpse Of Season 4 Drops", 'Trump Media stock rebounds 47% after brutal post-IPO sell-off', "NBA's exclusive TV rights negotiating window with ESPN, Warner expected to pass without a deal", 'Ally Financial Reports Upbeat Earnings, Joins Badger Meter, Genuine Parts And Other Big Stocks Moving Higher On Thursday - Ally Financial  ( NYSE:ALLY ) , Apogee Enterprises  ( NASDAQ:APOG ) ', 'Big-Picture Investing as Q1 Earnings Loom: Stocks to Watch', 'Netflix Is Enjoying An Upward Trajectory - But This Chart Shows Bearish Bias Ahead Of Q1 Earnings - Netflix  ( NASDAQ:NFLX ) ', "Netflix is set to report earnings - here's what Wall Street expects", "Netflix co-founder swears by 'great business philosophy' from Jeff Bezos: 'Take a lot of risks on things that are recoverable'", "The Crazy True Story Behind Netflix's 'Baby Reindeer'-What Happens To Martha?", 'What A TikTok Ban Would Mean For Tech And The Stock Market', 'Dow Jones Leads Rise; Nvidia Tests Critical Level', 'Initial Claims Lower Than Expected', 'Netflix Earnings Are Imminent; These Most Accurate Analysts Revise Forecasts Ahead Of Earnings Call - Netflix  ( NASDAQ:NFLX ) ', 'Jobless Claims Low & Steady, Philly Fed Surprises Higher', "Zack Snyder Says 'Four To Six' Rebel Moon Movies 'Makes Sense'", "Netflix Stock Is In The Spotlight Thursday: What's Going On? - Netflix  ( NASDAQ:NFLX ) ", 'Kartoon Studios Inc. Announces Pricing of up to $7 Million Registered Direct Offering of Common Stock - Kartoon Studios  ( AMEX:TOON ) ', "'Rebel Moon' On Netflix Stars Discuss The Parallels To Our Real World", 'Kartoon Studios Inc. Announces Pricing of up to $7 Million Registered Direct Offering of Common Stock', 'UNICEF USA BrandVoice: 6 Questions With UNICEF Ambassador Aria Mia Loberti', 'How Hollywood fell in love with video games', "Sony's Latest Bravia Series Elevates At-Home Streaming with Advanced Picture Settings and AI Features - Sony Group  ( NYSE:SONY ) ", 'Futures Rise Ahead Of Jobless Claims', 'Air Canada worker among 9 charged or sought over record airport gold heist', 'Netflix gained 9.3 million subscribers this year as it adjusts streaming strategy | Business', 'Netflix is the dominant streaming service. So why is it changing its model? | Business', "S&P 500, Nasdaq Futures Signal Rebound Today: What's Driving Stock Futures Higher? - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) , SPDR S&P 500  ( ARCA:SPY ) ", "Alibaba's Taobao launches 3D-capable app for use on Apple's Vision Pro headset", 'Zacks Industry Outlook Highlights Rogers Communication, Liberty Broadband and Liberty Global', 'The Most Valuable Combat Sports Promotions 2024', 'Why companies keep betting - and losing - on Harry and Meghan', 'Stock market today: Nasdaq 100 leads US futures in gains after AI chipmaker reports strong earnings overnight', 'Investor Sentiment Decreases Further, S&P 500 Falls For Fourth Straight Session - Marsh & McLennan  ( NYSE:MMC ) , D.R. Horton  ( NYSE:DHI ) ', 'D.R. Horton, Netflix And 3 Stocks To Watch Heading Into Thursday - Netflix  ( NASDAQ:NFLX ) , D.R. Horton  ( NYSE:DHI ) ', "Netflix's Earnings Day Is Here. The Focus Will Be on Subscriber Growth."], ['Fear the Walking Dead series two: What will happen to Nick?', 'Inside Kensington Palace, where Kate Middleton is isolating', 'How to watch Thursday Night Football for free: Eagles vs. Vikings live stream', "Disney, Charter's Spectrum deal will reshape media -executives", 'Disney Holds Talks To Sell ABC To Nexstar Media, Report Says', '19 pitch decks that helped media and tech startups raise millions to disrupt entertainment', 'You may live to be 100: What that means for your health, money and family', "Princess Diana's 'Black Sheep' Sweater Sells At Auction For $1.1 Mn", "Princess Diana's 'Black Sheep' Sweater Sells At Auction For $1.1 Mn", 'Netflix Faces Margin and Growth Equilibrium Challenges: Analyst - Netflix  ( NASDAQ:NFLX ) ', "This fall TV season, there's no new 'Grey's Anatomy' or 'Law & Order.' Here's what's coming instead", 'Netflix And Spotify Are Navigating The Streaming Crisis With Their Crowns Intact - Walt Disney  ( NYSE:DIS ) ', 'Stocks making the biggest moves midday: Netflix, Etsy, HP, Visa and more', 'Fox  ( FOXA )  Nation to Premiere Dahmer Family Tapes Docuseries', 'Audi Canada Welcomes Ethan Hawke for the 11th Installment of its homegrown Innovation Series', 'Audi Canada Welcomes Ethan Hawke for the 11th Installment of its homegrown Innovation Series - Canada NewsWire', "Adam Sandler is headed out on the 'I Missed You' comedy tour", "'Wrestlers' Music Composer The Secret Weapon In Telling Powerful Story Of OVW", "Forget Season 2, Netflix's 'One Piece' Is Mapping Out Six Seasons, Hoping For 12", '2 in 3 Parents Report Their Child Has Been Negatively Affected by Social Media', 'Netflix is well positioned for scripted programming in face of Hollywood strikes, analyst says', 'Korean Dramas Play A Role In The Booming Popularity Of Webtoons', 'Why Novo Integrated Sciences Shares Are Trading Higher By 9%; Here Are 20 Stocks Moving Premarket - Avalo Therapeutics  ( NASDAQ:AVTX ) , Biolase  ( NASDAQ:BIOL ) ', 'Read the 12-page pitch deck 2 brothers used to raise $2 million for Strada, an AI startup to streamline content creation', 'Microsoft Stock: Bear vs. Bull', "For better gains in tech stocks from here, look beyond the 'Magnificent Seven.'", "'Love at First Sight' Review: Sense, Sensibility and Statistics", 'Better Buy: Amazon Stock vs. Microsoft Stock', '3 Warren Buffett Stocks That Are Near Their 52-Week Lows', 'Top Boy review: a fitting finale to the gangland drama', 'Cable TV Is on Life Support, but a New Bundle Is Coming Alive', 'EY has created its own large-language model - and says it will train all 400,000 employees to use it as part of a $1.4 billion investment', 'Free-to-Air Services Market to Reach $337.3 Billion, Globally, by 2032 at 11.2% CAGR: Allied Market Research', 'Five ways David Bowie changed the world', 'Pangea Natural Foods Inc. Provides Comprehensive Corporate Update - PR Newswire', 'Pangea Natural Foods Inc. Provides Comprehensive Corporate Update - Canada NewsWire', 'Trai to float a consultation paper on regulations for OTT players soon', 'Peaky Blinders: when is season five and what can we expect?', 'Get Ready for Your Wedding, but Make It Glam', "Singapore is the world's 6th Blue Zone - and a look at its urban planning helps explain why", 'NFLX,FFIE,AMZN,AAL,TSLA: Top 5 Trending Stocks - Netflix  ( NASDAQ:NFLX ) , Tesla  ( NASDAQ:TSLA ) , Faraday Future  ( NASDAQ:FFIE ) , Amazon.com  ( NASDAQ:AMZN ) ', 'Wall Street mixed after inflation data as European bourses dip ahead of ECB', "Disney's Lost Decade and Next Act", "Game Developers Are 'Fucking Livid' Over Unity's New Fees", "Netflix's 'Full Swing' won't be getting full access to the Ryder Cup team rooms", 'Google to lay off hundreds of recruiting staffers', "Sean Penn says studio execs who won't agree to AI protections for actors should let him and his friends 'do whatever we want' with their daughters' likenesses", '6th Ottawa Korean Film Festival returns to Ottawa', 'Charter CFO Hails Disney Renewal as Triumph for Pay-TV Industry - Charter Communications  ( NASDAQ:CHTR ) , Walt Disney  ( NYSE:DIS ) ', 'Netflix Shares Tumble, Shed $9B In Value As CFO Lowers Margin Guidance - Netflix  ( NASDAQ:NFLX ) , Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) '], ['Bitget Wallet and Bitrefill Partner to Enable Crypto Payments for Gift Cards in 130 Countries', '2025 Goals: Invest Better, Budget Smarter', 'Is American Century U.S. Quality Growth ETF  ( QGRO )  a Strong ETF Right Now?', 'Can the 5 Worst-Performing Stock Market Sectors in 2024 Beat the S&P 500 in 2025?', "Disney's Ad-Supported Streaming Reaches 157 Million Users Globally, 112 Million In US: Here's How It Compares With Netflix And Amazon Prime - Amazon.com  ( NASDAQ:AMZN ) , Walt Disney  ( NYSE:DIS ) ", "Netflix Q4 To Show Advertising, Live Sports Strength, Analyst Says: 'Multiyear Narrative Of Compounded Growth' - Netflix  ( NASDAQ:NFLX ) ", 'Share the Swirl of Friendship with sweetFrog® and Care Bears™', 'How to Find Strong Consumer Discretionary Stocks Slated for Positive Earnings Surprises', '2 Unstoppable Streaming Stocks to Buy Hand Over Fist in 2025, According to Wall Street', "Fubo Gets 'Fairy Tale Outcome': Analyst Says Disney Deal Lowers Risks, Overhang On Stock - FuboTV  ( NYSE:FUBO ) ", 'IPSY Joins Forces With Gabrielle Union to Celebrate Black History Month', "What's Going On With TKO Group Holdings Stock Today? - TKO Group Holdings  ( NYSE:TKO ) ", 'BLACK ENTERPRISE Announces the 40 Under 40 List', "Dana White Wanted Elon Musk, Mark Zuckerberg In Cage Fight, Now He's On Meta's Board Of Directors - Meta Platforms  ( NASDAQ:META ) ", 'Netflix  ( NFLX )  Ascends But Remains Behind Market: Some Facts to Note', 'Investors Look Back at 2024', 'Netflix Brings WWE Monday Night RAW to Streaming Starting January 6 - Netflix  ( NASDAQ:NFLX ) ', "The World's Smallest Laser TV, the JMGO O2S Ultra, Shines at CES 2025", 'reVolver Podcasts--The Preferred Platform with Millions of Downloads--Launches Exclusive App on Samsung Galaxy Store', 'The Best Stocks to Invest $1,000 In Right Now', 'Comcast to launch Universal Ads in bid to win smaller advertisers over from tech', 'How the sports media world is undergoing a seismic shift', 'Melania Trump To Produce Upcoming Amazon Prime Documentary About Her Life and Influence In Trump Political Universe - Amazon.com  ( NASDAQ:AMZN ) , Walt Disney  ( NYSE:DIS ) ', "Wish You'd Bought Palantir in 2024? Buy This Unstoppable ETF Instead", "Consumer Tech News  ( Dec 30-Jan 3 ) : Apple Stops Selling iPhone 14, iPhone 14 Plus, iPhone SE, Trump Seeks A 'Political Resolution' Of TikTok Ban & More - Microsoft  ( NASDAQ:MSFT ) , Apple  ( NASDAQ:AAPL ) ", 'Tesla, Netflix, MicroStrategy - And Shiba Inu Strives To Outpace Dogecoin In Crypto: Bulls And Bears', '"Rule Breaker Investing" Mailbag: Portfolio-Level Thinking', 'Bearish Analyst Regrets Not Upgrading Netflix To Buy In 2023 - Netflix  ( NASDAQ:NFLX ) ', 'MGNI Stock Surges 31% in 3 Months: Is There Room for Further Growth?', 'Company News for Jan 3, 2025', 'Tesla To $351? Here Are 10 Top Analyst Forecasts For Friday - Citigroup  ( NYSE:C ) , Caesars Entertainment  ( NASDAQ:CZR ) ', '5 Dates for Disney Stock Investors to Circle in 2025']]
# For netflix gain - 
news_titles = [['News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business', 'News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business', 'T-Mobile Earnings Miss But Subscriber Adds Top AT&T, Verizon', 'Why Netflix Stock Was Soaring This Week', 'WWE boss Vince McMahon accused of sexual assault and trafficking in new lawsuit', 'Bear Call Spread In Netflix Adds $65 In Your Wallet Now', "Good News For 'Squid Game' Fans As Season 2 News Finally Surfaces", "Netflix Is Raising Prices In The Most Cynical Way And It's Only Going To Get Worse", "Here's How Much You Would Have Made Owning Netflix Stock In The Last 20 Years - Netflix  ( NASDAQ:NFLX ) ", 'Tesla: 5 Reasons the Earnings Dip is a Bargain', 'Netflix shares are fully priced, Wall Street analysts say. Here here are six other media stocks to think about.', 'Descendant Filmmaker Honored by Blacks In Government', 'Netflix Shares Surge Post Earnings: ETFs in Focus', 'Netflix  ( NFLX )  to Live Stream WWE RAW Exclusively Starting 2025', "How To Watch The WWII Miniseries 'Masters Of The Air'", 'Best Momentum Stocks to Buy for January 25th', 'Gradle Inc. Achieves Iron Bank Certification and Plans to Play a Key Role in Accelerating Software Delivery Time and Securing the DoD Software Supply Chain', 'Hughesnet Vs. Viasat: Which Is Best For You?', 'Why This 1 Momentum Stock Could Be a Great Addition to Your Portfolio', 'Business | Jan 27th 2024 Edition', 'Apple and Cenovus Energy have been highlighted as Zacks Bull and Bear of the Day', 'Superyacht Designers Bannenberg & Rowell Go BIG With 367-Foot-Long Renaissance', "From Scholar To Coder, Gamer To Seller: Here Are A Few Custom GPTs To Make Everyone's Life Easier - Microsoft  ( NASDAQ:MSFT ) ", 'Bell Fibe TV brings next generation TV and entertainment to Atlantic Canada', 'Bell Fibe TV brings next generation TV and entertainment to Atlantic Canada', 'Digital Player Affirm Holdings Ventures Into The Brick-And-Mortar World', 'Ally Invest Is Growing Its Name Among The Best Online Brokers', 'Futures: Analysts Slash Tesla Views After Earnings Call; Humana Dives', 'Nasdaq, S&P 500 Futures Unfazed By Tesla Disappointment As Focus Shifts To Q4 GDP, Earnings: Analyst Sees Hope In This Bullish Indicator - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) , SPDR S&P 500  ( ARCA:SPY ) ', 'Light shows: after-dark displays in the UK this winter', 'First Busey  ( NASDAQ:BUSE )  Upgraded to Hold at StockNews.com', '3 Reasons Roku Stock Could Crush the Market for the Next Decade', 'IQVIA  ( NYSE:IQV )  Upgraded to Buy by StockNews.com', 'Logitech International  ( NASDAQ:LOGI )  Upgraded to "Strong-Buy" at StockNews.com', 'UFP Technologies  ( NASDAQ:UFPT )  Downgraded to Sell at StockNews.com', 'Trustmark  ( NASDAQ:TRMK )  Downgraded by StockNews.com to Sell', 'Cirrus Logic  ( NASDAQ:CRUS )  Upgraded to "Buy" by StockNews.com', 'Franklin Resources  ( NYSE:BEN )  Upgraded to Buy at StockNews.com', 'Marcus  ( NYSE:MCS )  Upgraded to Buy by StockNews.com', 'EVERTEC  ( NYSE:EVTC )  Downgraded to Hold at StockNews.com', 'General Electric  ( NYSE:GE )  Downgraded by StockNews.com to Hold', 'Stride  ( NYSE:LRN )  Upgraded to "Buy" by StockNews.com', 'MakeMyTrip  ( NASDAQ:MMYT )  Upgraded to Hold by StockNews.com', 'Miller Industries  ( NYSE:MLR )  Downgraded to "Buy" at StockNews.com', 'Boyd Gaming  ( NYSE:BYD )  Upgraded to Buy by StockNews.com', 'International Game Technology  ( NYSE:IGT )  Upgraded to "Strong-Buy" by StockNews.com', 'Altisource Asset Management  ( NYSE:AAMC )  Coverage Initiated by Analysts at StockNews.com', 'CSP  ( NASDAQ:CSPI )  Lowered to "Hold" at StockNews.com', 'Intrepid Potash  ( NYSE:IPI )  Upgraded to "Sell" at StockNews.com', 'Quest Diagnostics  ( NYSE:DGX )  Upgraded to "Buy" by StockNews.com'], ['WEBTOON Entertainment Inc. to Participate in Upcoming Investor Conferences', 'WEBTOON Entertainment Inc. to Participate in Upcoming Investor Conferences - Webtoon Entertainment  ( NASDAQ:WBTN ) ', "Does Netflix's Push Into Live Steaming Justify Buying the Stock Now?", 'Cineverse Releases Holiday Movie Slate to Air on The Dove Channel - Cineverse  ( NASDAQ:CNVS ) ', 'Dinewise, Inc.  ( DWIS )  Appoints Former NFL Star Michael Farr as CEO', 'Asset Entities Signs Agreement with Netflix Star Jas Leverette to Build Digital Dog Training Community - Asset Entities  ( NASDAQ:ASST ) ', 'SPACE FOR HUMANITY EXPRESSES SUPPORT FOR SCIENCE COMMUNICATOR HEADING TO SPACE', "3 Unstoppable Stocks With Competitive Moats That Appear Poised to Become Wall Street's Next Stock-Split Stocks in 2025", "Mohamed El-Erian Warns Against Simplistic Narratives As Trump Plans Aggressive Tariff Strategy: 'The Issue Is Quite Complex' - Applied Mat  ( NASDAQ:AMAT ) , Broadcom  ( NASDAQ:AVGO ) ", 'Unmissable Black Friday Deals on HAPPRUN Projectors - Transform Your Home Theater!', 'RUSSELL PETERS NEW STAND-UP SPECIAL: ACT YOUR AGE Live in Abu Dhabi PREMIERING ON PATREON DECEMBER 5TH, 2024 AT 6PM PST', 'FOX vs. NFLX: Which Stock Should Value Investors Buy Now?', "AMD's HPC Portfolio Powers El Capitan: How Should You Play the Stock?", 'Warner Bros. Discovery Falls 17% YTD: How Should You Play the Stock?', 'The Smurf Experience - A 13,000-Square-Foot Family Attraction - Sets its U.S. Debut for Early 2025', 'Unpacking the Latest Options Trading Trends in Netflix - Netflix  ( NASDAQ:NFLX ) ', 'Atmospheric Scientist Katharine Hayhoe to Discuss Finding Hope in the Face of Climate Change During Public Talk at Dickinson College', 'Qloo® Partners with Special Olympics to Personalize Donor Engagement and Identify New Donors Through Taste AI™', 'Athletic Brewing and Netflix Launch Marine Odyssey IPA', "Bitcoin and Dogecoin Rise While Ethereum Drops; BlackRock's ETF Hits $1B Volume; Walmart Market Share Soars Driven by Six-Figure Income Households - Top Headlines Today While US Slept", 'Netflix Made DVDs Obsolete By Popularizing Streaming. Ben Affleck Thinks AI Will Change The Face Of Hollywood Next - Netflix  ( NASDAQ:NFLX ) , Walt Disney  ( NYSE:DIS ) ', 'TransPerfect Media Named Netflix Preferred Fulfillment Partner of the Year 2024 for APAC Region', '2 Hypergrowth Tech Stocks to Buy in 2024 and Beyond', 'The Future is Here: New Mexico Media Arts Collective Launches First-of-its-Kind Emerging Technology Lecture Series', "Disney's Q4 Earnings Surpass Estimates: How to Play the Stock", "Disney's Stock Is Too Cheap to Pass Up", 'Disney Investors Just Got Fantastic News, but Is the Stock a Buy?', 'Billionaire Jerry Jones Just Explained Why Live Sports Is a No-Brainer for Netflix. Is the Streaming Stock About to Pop Again?', 'Brokers Suggest Investing in Netflix  ( NFLX ) : Read This Before Placing a Bet', "Tesla Is Back In The Future Fund Active ETF's Top 5 Holdings - Tesla  ( NASDAQ:TSLA ) ", 'Netflix Stock Soars Amid Tyson-Paul Boxing Stream Stutters; Analyst Gary Black Says Viewership Signals Q4 Subscriber Growth', 'Warner Bros. Secures 11-Year NBA Deal Extension - Warner Bros. Discovery  ( NASDAQ:WBD ) ', 'Netflix Crash Hits Jake Paul-Mike Tyson Fight: Can Streaming Platform Handle Christmas Day NFL/Beyonce Action? - Netflix  ( NASDAQ:NFLX ) ', 'Netflix Scores Big: Beyoncé Set To Headline NFL Christmas Halftime Show - Netflix  ( NASDAQ:NFLX ) ', 'Disney Shines In All Its Entertainment Glory - Walt Disney  ( NYSE:DIS ) ', "LeBron James' SpringHill to merge with 'The Kardashians' producer Fulwell 73", 'MULTIMEDIA UPDATE -- Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health - Prenetics Global  ( NASDAQ:PRE ) ', 'MULTIMEDIA UPDATE -- Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health', 'Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health', 'Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health - Prenetics Global  ( NASDAQ:PRE ) ', "Warner's Max launches in Asia, taking on Netflix in Hong Kong", 'Mike Tyson Discloses Near-Death Experience Before Jake Paul Fight', 'The Future of Live Sports on Netflix: Lessons Learned From Problems With the Tyson-Paul Event', "Jake Paul Vs. Mike Tyson Boxing Match Sets 'Record-Breaking Night' For Netflix: Streaming Giant Says Nearly 60M Households Tuned In - Netflix  ( NASDAQ:NFLX ) ", 'The Best Stocks to Invest $50,000 in Right Now', "Mike Tyson Vs. Jake Paul: Netflix's Explosive Boxing Event Ends With Paul's Victory - Netflix  ( NASDAQ:NFLX ) ", "The Trade Desk's Valuation Looks Expensive. Can the Stock Outperform in 2025?", 'Time to Buy Home Depot or Disney Stock After Beating Earnings Expectations?', 'Jake Paul, Mike Tyson fight breaks record for biggest boxing gate outside of Las Vegas', "Netflix Streams Mike Tyson Vs. Jake Paul: The Fight Everyone's Talking About - Netflix  ( NASDAQ:NFLX ) "], ['Is Netflix Stock Poised to Grow Again?', 'What Comes Next In The Ukraine War--Plus, Fading The Recent Rally With Bets Against Real Estate - Netflix  ( NASDAQ:NFLX ) ', "Why Netflix Won't Forecast Subscriber Numbers Anymore", 'Video: Formula 1 CEO Stefano Domenicali Talks U.S. Grand Prix, Staying In Austin Long-Term', 'Why Upstart, PayPal, and SoFi Stocks Were Volatile Today', 'Is Netflix Password Sharing An Even Bigger Problem Than People Thought?', 'Are All Streaming Services Losing Money? Netflix Seems To Think So - Netflix  ( NASDAQ:NFLX ) ', "WACO CENTER'S WEARABLE ART GALA RETURNS WITH STEP INTO HARLEM NIGHTS", "India's top multiplex chains bank on festive season to bring cheer back", 'Why Marqeta Stock Jumped Again Today', "India's top multiplex chains bank on festive season to bring cheer back", 'Education Department could flag up to 5 million student loan relief applications for review - what to do if yours is among them', 'FUNRISE, INC. AND PARAMOUNT CONSUMER PRODUCTS PARTNER TO LAUNCH TEENAGE MUTANT NINJA TURTLES TOY CARS FALL 2023 - Paramount Global  ( NASDAQ:PARA ) ', 'CIO Leadership: The Modern Technology Executive - Leading, Inventing and Winning in Unprecedented Times and Uncharted Waters Will Drive the Discussion at the 2022 Greenwich CIO Executive Leadership Summit on November 3', 'AT&T;  ( T )  Q3 2022 Earnings Call Transcript', "Movie theaters want more from Netflix, but the streaming giant isn't ready to budge on its release model", "Box Office: 'Black Panther: Wakanda Forever' Aims For Record $175 Million Debut", 'The Best New Movies And Shows On Netflix Today: October 20, 2022', 'Wall Street rises as upbeat corporate forecasts trump UK political shock', 'Why Is Disney Stock Down 37% This Year?', 'Higher Rates vs. Q3 Earnings - Blackstone  ( NYSE:BX ) , FTX Token  ( FTT/USD ) ', 'Business', 'Stock Market News for Oct 20, 2022', "I've Been a Netflix Shareholder for 20 Years", "US Stocks: S&P 500 to open lower as UK's Truss resigns as PM | The Financial Express", 'Netflix Stock Surges on Earnings Beat and Subscriber Growth: 6 Metrics You Should See', 'Better Buy: Coca-Cola vs. Disney', 'Is Netflix Stock a Buy Now?', 'Haas F1 inks MoneyGram to 3-year title sponsorship deal', 'Futures Rise; Tesla Skids On Sales, Musk', "This Analyst With 81% Accuracy Rate Upgrades Netflix; Here Are 4 Other Stock Picks For October 19 From 's Most Accurate Analysts - Carvana  ( NYSE:CVNA ) , Equitable Holdings  ( NYSE:EQH ) ", "Warner Bros Releases Ethereum-Compatible 'Lord of the Rings' NFTs", 'The Zacks Analyst Blog Highlights Netflix, Intuitive Surgical, Lockheed Martin and United Airlines', 'Nasdaq futures fall as Tesla warns on 2022 deliveries', '3 Explosive Stocks to Buy Right Now', 'Weekender: 25 things to do in Charlotte this weekend including pumpkin carving and chili tasting', 'Intuitive Surgical, Netflix, ASML And Other Big Gainers From Wednesday - Amprius Technologies  ( NYSE:AMPX ) , ASML Holding  ( NASDAQ:ASML ) ', 'Zacks Investment Ideas feature highlights: Netflix, Microsoft and Disney', 'Investor Fear Eases Slightly Ahead Of Earnings Results - American Airlines Group  ( NASDAQ:AAL ) , Amazon.com  ( NASDAQ:AMZN ) ', 'Talenthouse to partner with and invest up to $7 million in cash and shares in US-based digital SME banking provider, Nerve.', 'Talenthouse to partner with and invest up to $7 million in cash and shares in US-based digital SME banking provider, Nerve.', 'Global stocks lower amid British political turmoil', 'Asian shares fall after weak earnings pull Wall St lower', 'Asian Markets Drop And Dollar Rises As Inflation, Rate Fears Return', 'Asian Markets Drop And Dollar Rises As Inflation, Rate Fears Return', 'Equities close lower as rise in yields overshadows earnings', 'While Netflix Pushes Quantity, Quality Matters More To Most Viewers, UTA Data Suggests', "Buy shares of Netflix on the market's next pullback, Jim Cramer says", 'Why Roku Stock Popped Then Dropped on Wednesday', "Controversy Over 'The Crown' Grows: Judi Dench Says Netflix Should Add Disclaimers To Series"], ['NFL Games Set Thanksgiving Day Records: Why This Is Good And Bad News For Streaming Giant Netflix - Netflix  ( NASDAQ:NFLX ) ', 'Trakt Partners with Younify to launch its Streaming Scrobbler', "Aaron Rodgers Opens Up About Psychedelic Healing In New Netflix Docuseries, 'Enigma'", "Netflix Stock Reaches New All-Time High This Week: What's Driving The Surge? - Netflix  ( NASDAQ:NFLX ) ", 'Is Netflix  ( NFLX )  a Buy as Wall Street Analysts Look Optimistic?', "MasterClass, Women's Health Advocate Halle Berry and Leading Health Experts Come Together for Stigma-Breaking Class on Menopause", 'Toyota Spreads Festive Cheer This Holiday Season - Toyota Motor  ( NYSE:TM ) ', '1 No-Brainer Vanguard Sector ETF to Buy Right Now for Less Than $500', 'The Top 16 Stocks to Buy Now in December  ( 2024 ) ', '3 Breakout Growth Stocks You Can Buy and Hold for the Next Decade', 'Netflix  ( NFLX )  Surpasses Market Returns: Some Facts Worth Knowing', 'Roku Shares Are On The Rise Today: What You Need To Know - Roku  ( NASDAQ:ROKU ) ', 'The Trade Desk Just Made a Big Move That Has Investors Scrambling to Buy Roku Stock', "Peacock Network to Premiere Visit Myrtle Beach's TV Series 'Traveling the Spectrum' on Dec. 12", 'Netflix Tops 10 Million Subscribers In Japan - Netflix  ( NASDAQ:NFLX ) ', '5 Stocks Driving Nasdaq ETF to New Highs in 2024', 'Traditional Pay-TV Subscriptions to Dip Below 35% in 2025 as Streaming Becomes the Norm - AlixPartners Annual Media Predictions', 'Top 3 Tech And Telecom Stocks That May Implode This Quarter - Meta Platforms  ( NASDAQ:META ) , Walt Disney  ( NYSE:DIS ) ', 'Is Bluesky the Killer App That Will Kickstart the Web3 Revolution in 2025?', 'Up 137%, Should This Red-Hot Artificial Intelligence Cryptocurrency Be in Your Portfolio for 2025?', 'Should You Buy Amazon Stock Before Dec. 19?', 'Consumer Electronics Manufacturing Market to Surge Valuation Set to Hit US$ 4,267.27 Billion by 2034 at 9.5% CAGR | Fact.MR Report', "'Squid Game' Season 2 Includes New Games, Contestant With A Crypto Background, Revenge Factor, Political Overtones - Netflix  ( NASDAQ:NFLX ) ", 'Verizon turns up the heat on savings with Holiday Deal Days', 'Rule Breaker Investing: The 2024 Gratitude Episode', 'Company News for Dec 3, 2024', 'SCHOLASTIC UNLEASHES DOG MAN GLOBAL CAMPAIGN CELEBRATING READING AND HEROES AROUND THE WORLD - Scholastic  ( NASDAQ:SCHL ) ', 'TWLO Enhances Its Segment Platform: What Should Investors Do?', "Domino's® and Netflix Partner to Bring Emergency Pizza to the Squid Game Universe - Netflix  ( NASDAQ:NFLX ) , Domino's Pizza  ( NYSE:DPZ ) ", "This Vanguard ETF Is Up 32% in 2024. Here's Why It's a Simple Way to Invest in Google Parent Alphabet, Meta Platforms, Netflix, and Disney.", 'From Shopping to Advice From Warren Buffett', "How Dick's Sporting Goods Is Succeeding", "Macy's Multimillion-Dollar Error", 'Trump\'s Return to the White House Could Spark Unprecedented Investment Opportunity: Experts Eye Starlink\'s Potential "Super-IPO"', 'Netflix To Rally More Than 7%? Here Are 10 Top Analyst Forecasts For Monday - Applied Therapeutics  ( NASDAQ:APLT ) , Affirm Holdings  ( NASDAQ:AFRM ) ', 'Costco Sales Topped $250 Billion in Fiscal 2024. But Less Than 2% of Those Sales Are Key to Understanding the Entire Business.', 'Could Buying Nu Stock Today Set You Up for Life?', 'If I Could Only Buy and Hold a Single Stock, This Would Be It', 'Roku  ( ROKU )  Up 5.7% Since Last Earnings Report: Can It Continue?', 'Is the iShares Expanded Tech Sector ETF a Millionaire Maker?', "Is fuboTV Stock a Buy Following Netflix's Live Sporting Event Debacle?"], ['Morning Bid: Five alive: US yield curve nears historic level', 'Stock Market Today: Powell Sends Stocks On Roller-Coaster Ride', "US stocks fall as the 10-year US Treasury yield approaches 5% and Powell says inflation 'still too high'", "Netflix price increase: Here's how much the major streaming services are set to cost", 'Dow Jones Dives Amid Powell Pledge; Tesla Sinks After Elon Musk Warning', 'Netflix Thrived Despite Hollywood Tensions - Netflix  ( NASDAQ:NFLX ) ', "As Netflix's stock heads for best one-day in almost three years, investors are snapping up its bonds", 'Netflix Hikes Prices Again As Subscriptions Climb', 'A travel influencer shared a plethora of tips on how to best sleep in airports, including chaining luggage to your body with earbud wires', 'Netflix Rallies Off Q3 Earnings: 8 Analysts On Future Price Increases, Ad-Supported Plans And More - Netflix  ( NASDAQ:NFLX ) ', 'NFLX: 3 Internet Stocks to Keep Watching for Future Gains', 'Dow Jumps 150 Points; Netflix Earnings Top Views - SeaStar Medical Holding  ( NASDAQ:ICU ) , Lindsay  ( NYSE:LNN ) ', 'Wall Street Cheers Netflix Price Increases As Sign Of Strength', "Five In The Wallet: 10-Year Yield Flirts With 5% For First Time Since 2007 As Fed's Powell Speech Awaited - CSX  ( NASDAQ:CSX ) , American Airlines Group  ( NASDAQ:AAL ) ", 'Netflix  ( NFLX )  Q3 Earnings Beat, Revenues Rise Y/Y on User Gain', 'Why Netflix Stock Jumped Back Over $400 Per Share Today', 'Netflix Shares Surge 15% After Subscriber Growth-Largest Single-Day Gain Since 2021', 'Tesla & Netflix Earnings: A Closer Look', 'Why Streaming Service Roku Inc Stock Is Up Today - Netflix  ( NASDAQ:NFLX ) , Roku  ( NASDAQ:ROKU ) ', 'Stock Market Erases Gains And Here Comes Powell; Tesla Stock Has This Problem', 'Market Clubhouse Morning Memo - October 19th, 2023  ( Trade Strategy For SPY, QQQ, AAPL, MSFT, NVDA, TSLA, And NFLX )  - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) ', 'Stocks Mixed After Jobless Claims', 'Netflix stock surges 16% after Wall Street buys into ad-driven subscriber growth', "ETFs to Buy on Netflix's Blowout Q3 Earnings", 'Tesla, Netflix Earnings: Everything You Need to Know', 'AT&T, Union Pacific, Las Vegas Sands And Other Big Stocks Moving Higher On Thursday - American Airlines Group  ( NASDAQ:AAL ) , Badger Meter  ( NYSE:BMI ) ', "US stocks rise as key Treasury yield nears 5% ahead of Powell's speech", "Hollywood's actors strike is nearing its 100th day. Why hasn't a deal been reached and what's next?", 'Netflix Stock: Bear vs. Bull', "Netflix Stock Fast Forwards As Subscribers Surge: What's Going On Thursday? - Netflix  ( NASDAQ:NFLX ) ", 'The top 10 things to watch in the stock market Thursday', 'Netflix Realigns Pricing Strategy In Australia, Ends Basic Plan For New Members - Netflix  ( NASDAQ:NFLX ) ', 'Netflix Stock Jumps 13% on Earnings Beat and Strong Subscriber Growth', "Martin Scorsese's 'Killers of the Flower Moon' wants to be epic but just feels long", 'Where to look to find enlightenment', 'Business | Oct 21st 2023 Edition', 'The daily business briefing: October 19, 2023', 'Best Buy To Rally More Than 21%? Here Are 10 Top Analyst Forecasts For Thursday - First Solar  ( NASDAQ:FSLR ) , Best Buy Co  ( NYSE:BBY ) ', 'Why Netflix Shares Are Trading Higher By Around 14%; Here Are 20 Stocks Moving Premarket - Biocept  ( NASDAQ:BIOC ) , Atreca  ( NASDAQ:BCEL ) ', "Netflix's Ads Business Is Taking Off", "Futures inch up ahead of Powell's comments; Tesla drops, Netflix surges", 'Netflix adds $28 billion in market value after password crackdown drives subscriber spike', "Netflix's total valuation is set to balloon by over $20 billion after the streaming giant said subscriber numbers had soared", 'Netflix adds $28 billion in market value after password crackdown drives subscriber spike', "S&P 500, Nasdaq Set For Mixed Open After Tesla, Netflix Earnings: What's Moving Stock Futures? - Alaska Air Gr  ( NYSE:ALK ) , Netflix  ( NASDAQ:NFLX ) , VMware  ( NYSE:VMW ) , Broadcom  ( NASDAQ:AVGO ) ", 'Stanley Druckenmiller said central banks, not earnings, move markets. Today is the day to pay attention.', '5 things to know for Oct. 19: Israel, House speaker, Covid-19, Pollution, UFOs', 'Futures: Yields Eye 5%; Tesla Falls On These Elon Musk Warnings', 'Morning Bid: To 5% and beyond, bond yields soar', "Frasier review: a 'gentle' reboot of the beloved 1990s sitcom"]]
news_titles = list(itertools.chain.from_iterable(news_titles))
news_titles_appended = ""
for news_title in news_titles:
    news_titles_appended = news_titles_appended + "," + "\"" + news_title + "\""

messages = [
    {"role": "user", "content": user_prompt_for(news_titles_appended)},
    {"role": "system", "content": system_prompt}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response)
print(response.json()['message'])
print(response.json()['message']['content'])

<Response [200]>
{'role': 'assistant', 'content': ''}



In [172]:
# news_titles = [[], ["Here's everything new coming to HBO Max in May 2022 -- and what's leaving", 'Why Netflix Stock Flopped Again Today', 'Disney stock tumbles amid Florida bill controversy', 'Dow falls 368 points as Federal Reserve hints at raising interest rates', "If You Invested $1,000 In Netflix The Last Time The Company Lost Subscribers, Here's How Much You'd Have Now", '3 Reasons to Buy Netflix, and 1 Reason to Sell', 'CNN+ to shut down by end of April due to new management decision', "DraftKings Stock Falls To 52-Week Low: What's Going On?", "For Ackman's Pershing Square, a Record of Big Hits and Misses", 'Markets Fall As Investors Weigh Earnings', 'Dow Jones Falls As Powell Says This On Inflation; Elon Musk Makes Twitter Move; Tesla Stock Gains Fade', 'CNN+ Failed for the Reason Netflix Struggles and Disney+ Thrives', "Real estate ETFs see these types of properties holding up in 'tricky' time for investing", 'Roth IRAs, ESG Stocks, Netflix: Community Conversations', "CNN's streaming service shutting down a month after launch", "These Netflix Analysts Aren't Confident Of A Turnaround Within The Next 18 Months", 'KeyCorp  ( KEY )  Q1 2022 Earnings Call Transcript', 'Boat Rocker Media Announces Details of Q1 2022 Financial Results Conference Call', 'KR: Think the Market is In for a Rough Ride? Then Add These 5 Stocks to Your Portfolio', "Nvidia Stock Is Sliding: What's Going On?", 'Roughly 1 In 10 Adults Use Netflix Password From Another Household-With Baby Boomers The Biggest Culprits-Survey Suggests', 'What Are Whales Doing With Netflix', 'Why Nvidia Stock Fell Hard Today', 'Is AT&T Stock Back in Action and Worth Buying Now?', 'Dow Drops Over 300 Points, Yields Surge After Powell Signals Bigger Rate Hikes Starting In May', 'Carvana Soared but Now Falls Behind Auto-Dealer Rivals', 'HBO Max And HBO Rise To 76.8M Global Subscribers', "Searches for 'Buy Netflix Stock' Explode 931% Following Share Price Plummet For Streaming Giant", "Maybe Netflix's Problem is Its Shows, Not Password Sharing", 'Streaming TV With Ads Puts Netflix And Its Peers At A Crossroads', 'Will DraftKings Inc.  ( DKNG )  Be On Your List Of Penny Stocks In 2022?', 'AT&T;  ( T )  Q1 2022 Earnings Call Transcript', 'Stocks That Hit 52-Week Lows On Thursday', "CNN's streaming service shutting down a month after launch", 'Move Over Netflix? HBO Max Is Gaining Subscribers And Market Share', "CNN's streaming service shutting down a month after launch", "Netflix Delivers Verdict On Streaming's Future: It's TV, Stupid", "10 Stocks Moved By Traders On Thursday's CNBC's 'Fast Money: Halftime Report'", 'Trading Tesla Stock After Impressive Earnings Beat', 'Stocks making the biggest moves midday: United Airlines, AT&T, Tesla and more', 'OnePlus TV 43 Y1S Pro review: Style and substance on a budget | The Financial Express', "Disney Stock Provides Meat For The Bears: Here's Where To Watch For The Bounce", "'Doctor Strange,' 'Thor' And 'Jurassic World' Sequels Top Fandango Summer Movie Poll", 'Better Buy: Netflix vs. Twitter', 'DARK & LOVELY AND STORM REID HOST BUILDING BEAUTIFUL FUTURES MASTERCLASS: "A BLACK GIRL\'S GUIDE TO MENTORSHIP"', 'Transportation: Strong Earnings Form Railroad And Airline Stocks Boost the Sector', 'Netflix Looks to 100 Million Existing Viewers to Boost Revenue', 'U.S. Stocks: Wall Street rises on strong earnings from Tesla, airlines | The Financial Express', 'US stocks rise on upbeat earnings', 'This 26-year-old Google engineer earns $270,000 a year and still uses coupons'], ['Big Q1 Beats After the Close; Markets Struggle to Gain', 'Netflix forces Wall Street to focus on profit and revenue with decision to stop reporting subscriber numbers in 2025', "Netflix keeps adding tons of subscribers from its password-sharing crackdown - and it's good news for Disney", 'Netflix is going to stop telling the world how many subscribers it has. Why?', 'Netflix  ( NFLX )  Beats Q1 Earnings and Revenue Estimates', 'Netflix now has nearly 270 million subscribers after another strong showing to begin 2024', 'Market About To Break Power Trend; Netflix Users Jump', 'Netflix Beats Q1 Targets But Comes Up Short With Q2 Sales Outlook', 'Netflix Q1 Earnings Highlights: Revenue Beat, EPS Beat, 9.33 Million Paid Subs Added, Ad-Tier Plan Update And More - Netflix  ( NASDAQ:NFLX ) ', 'Netflix smashed expectations, adding way more subscribers than Wall Street could have hoped for', 'Trump Media stock rebounds 47% after brutal post-IPO sell-off', "Caitlin Clark salary criticisms a 'false narrative,' says WNBA commissioner Cathy Engelbert", 'Stock market today: Stocks slide for 5th session as Fed officials cast more doubt on rate cuts', "Netflix's 'Baby Reindeer' Is An Incredibly Bizarre, Brilliantly Told True Story", "Britain's Prince Harry formally confirms he is now a US resident", "Netflix Renews And Also Cancels 'The Witcher' As First Glimpse Of Season 4 Drops", 'Trump Media stock rebounds 47% after brutal post-IPO sell-off', "NBA's exclusive TV rights negotiating window with ESPN, Warner expected to pass without a deal", 'Ally Financial Reports Upbeat Earnings, Joins Badger Meter, Genuine Parts And Other Big Stocks Moving Higher On Thursday - Ally Financial  ( NYSE:ALLY ) , Apogee Enterprises  ( NASDAQ:APOG ) ', 'Big-Picture Investing as Q1 Earnings Loom: Stocks to Watch', 'Netflix Is Enjoying An Upward Trajectory - But This Chart Shows Bearish Bias Ahead Of Q1 Earnings - Netflix  ( NASDAQ:NFLX ) ', "Netflix is set to report earnings - here's what Wall Street expects", "Netflix co-founder swears by 'great business philosophy' from Jeff Bezos: 'Take a lot of risks on things that are recoverable'", "The Crazy True Story Behind Netflix's 'Baby Reindeer'-What Happens To Martha?", 'What A TikTok Ban Would Mean For Tech And The Stock Market', 'Dow Jones Leads Rise; Nvidia Tests Critical Level', 'Initial Claims Lower Than Expected', 'Netflix Earnings Are Imminent; These Most Accurate Analysts Revise Forecasts Ahead Of Earnings Call - Netflix  ( NASDAQ:NFLX ) ', 'Jobless Claims Low & Steady, Philly Fed Surprises Higher', "Zack Snyder Says 'Four To Six' Rebel Moon Movies 'Makes Sense'", "Netflix Stock Is In The Spotlight Thursday: What's Going On? - Netflix  ( NASDAQ:NFLX ) ", 'Kartoon Studios Inc. Announces Pricing of up to $7 Million Registered Direct Offering of Common Stock - Kartoon Studios  ( AMEX:TOON ) ', "'Rebel Moon' On Netflix Stars Discuss The Parallels To Our Real World", 'Kartoon Studios Inc. Announces Pricing of up to $7 Million Registered Direct Offering of Common Stock', 'UNICEF USA BrandVoice: 6 Questions With UNICEF Ambassador Aria Mia Loberti', 'How Hollywood fell in love with video games', "Sony's Latest Bravia Series Elevates At-Home Streaming with Advanced Picture Settings and AI Features - Sony Group  ( NYSE:SONY ) ", 'Futures Rise Ahead Of Jobless Claims', 'Air Canada worker among 9 charged or sought over record airport gold heist', 'Netflix gained 9.3 million subscribers this year as it adjusts streaming strategy | Business', 'Netflix is the dominant streaming service. So why is it changing its model? | Business', "S&P 500, Nasdaq Futures Signal Rebound Today: What's Driving Stock Futures Higher? - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) , SPDR S&P 500  ( ARCA:SPY ) ", "Alibaba's Taobao launches 3D-capable app for use on Apple's Vision Pro headset", 'Zacks Industry Outlook Highlights Rogers Communication, Liberty Broadband and Liberty Global', 'The Most Valuable Combat Sports Promotions 2024', 'Why companies keep betting - and losing - on Harry and Meghan', 'Stock market today: Nasdaq 100 leads US futures in gains after AI chipmaker reports strong earnings overnight', 'Investor Sentiment Decreases Further, S&P 500 Falls For Fourth Straight Session - Marsh & McLennan  ( NYSE:MMC ) , D.R. Horton  ( NYSE:DHI ) ', 'D.R. Horton, Netflix And 3 Stocks To Watch Heading Into Thursday - Netflix  ( NASDAQ:NFLX ) , D.R. Horton  ( NYSE:DHI ) ', "Netflix's Earnings Day Is Here. The Focus Will Be on Subscriber Growth."], ['Fear the Walking Dead series two: What will happen to Nick?', 'Inside Kensington Palace, where Kate Middleton is isolating', 'How to watch Thursday Night Football for free: Eagles vs. Vikings live stream', "Disney, Charter's Spectrum deal will reshape media -executives", 'Disney Holds Talks To Sell ABC To Nexstar Media, Report Says', '19 pitch decks that helped media and tech startups raise millions to disrupt entertainment', 'You may live to be 100: What that means for your health, money and family', "Princess Diana's 'Black Sheep' Sweater Sells At Auction For $1.1 Mn", "Princess Diana's 'Black Sheep' Sweater Sells At Auction For $1.1 Mn", 'Netflix Faces Margin and Growth Equilibrium Challenges: Analyst - Netflix  ( NASDAQ:NFLX ) ', "This fall TV season, there's no new 'Grey's Anatomy' or 'Law & Order.' Here's what's coming instead", 'Netflix And Spotify Are Navigating The Streaming Crisis With Their Crowns Intact - Walt Disney  ( NYSE:DIS ) ', 'Stocks making the biggest moves midday: Netflix, Etsy, HP, Visa and more', 'Fox  ( FOXA )  Nation to Premiere Dahmer Family Tapes Docuseries', 'Audi Canada Welcomes Ethan Hawke for the 11th Installment of its homegrown Innovation Series', 'Audi Canada Welcomes Ethan Hawke for the 11th Installment of its homegrown Innovation Series - Canada NewsWire', "Adam Sandler is headed out on the 'I Missed You' comedy tour", "'Wrestlers' Music Composer The Secret Weapon In Telling Powerful Story Of OVW", "Forget Season 2, Netflix's 'One Piece' Is Mapping Out Six Seasons, Hoping For 12", '2 in 3 Parents Report Their Child Has Been Negatively Affected by Social Media', 'Netflix is well positioned for scripted programming in face of Hollywood strikes, analyst says', 'Korean Dramas Play A Role In The Booming Popularity Of Webtoons', 'Why Novo Integrated Sciences Shares Are Trading Higher By 9%; Here Are 20 Stocks Moving Premarket - Avalo Therapeutics  ( NASDAQ:AVTX ) , Biolase  ( NASDAQ:BIOL ) ', 'Read the 12-page pitch deck 2 brothers used to raise $2 million for Strada, an AI startup to streamline content creation', 'Microsoft Stock: Bear vs. Bull', "For better gains in tech stocks from here, look beyond the 'Magnificent Seven.'", "'Love at First Sight' Review: Sense, Sensibility and Statistics", 'Better Buy: Amazon Stock vs. Microsoft Stock', '3 Warren Buffett Stocks That Are Near Their 52-Week Lows', 'Top Boy review: a fitting finale to the gangland drama', 'Cable TV Is on Life Support, but a New Bundle Is Coming Alive', 'EY has created its own large-language model - and says it will train all 400,000 employees to use it as part of a $1.4 billion investment', 'Free-to-Air Services Market to Reach $337.3 Billion, Globally, by 2032 at 11.2% CAGR: Allied Market Research', 'Five ways David Bowie changed the world', 'Pangea Natural Foods Inc. Provides Comprehensive Corporate Update - PR Newswire', 'Pangea Natural Foods Inc. Provides Comprehensive Corporate Update - Canada NewsWire', 'Trai to float a consultation paper on regulations for OTT players soon', 'Peaky Blinders: when is season five and what can we expect?', 'Get Ready for Your Wedding, but Make It Glam', "Singapore is the world's 6th Blue Zone - and a look at its urban planning helps explain why", 'NFLX,FFIE,AMZN,AAL,TSLA: Top 5 Trending Stocks - Netflix  ( NASDAQ:NFLX ) , Tesla  ( NASDAQ:TSLA ) , Faraday Future  ( NASDAQ:FFIE ) , Amazon.com  ( NASDAQ:AMZN ) ', 'Wall Street mixed after inflation data as European bourses dip ahead of ECB', "Disney's Lost Decade and Next Act", "Game Developers Are 'Fucking Livid' Over Unity's New Fees", "Netflix's 'Full Swing' won't be getting full access to the Ryder Cup team rooms", 'Google to lay off hundreds of recruiting staffers', "Sean Penn says studio execs who won't agree to AI protections for actors should let him and his friends 'do whatever we want' with their daughters' likenesses", '6th Ottawa Korean Film Festival returns to Ottawa', 'Charter CFO Hails Disney Renewal as Triumph for Pay-TV Industry - Charter Communications  ( NASDAQ:CHTR ) , Walt Disney  ( NYSE:DIS ) ', 'Netflix Shares Tumble, Shed $9B In Value As CFO Lowers Margin Guidance - Netflix  ( NASDAQ:NFLX ) , Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) '], ['Bitget Wallet and Bitrefill Partner to Enable Crypto Payments for Gift Cards in 130 Countries', '2025 Goals: Invest Better, Budget Smarter', 'Is American Century U.S. Quality Growth ETF  ( QGRO )  a Strong ETF Right Now?', 'Can the 5 Worst-Performing Stock Market Sectors in 2024 Beat the S&P 500 in 2025?', "Disney's Ad-Supported Streaming Reaches 157 Million Users Globally, 112 Million In US: Here's How It Compares With Netflix And Amazon Prime - Amazon.com  ( NASDAQ:AMZN ) , Walt Disney  ( NYSE:DIS ) ", "Netflix Q4 To Show Advertising, Live Sports Strength, Analyst Says: 'Multiyear Narrative Of Compounded Growth' - Netflix  ( NASDAQ:NFLX ) ", 'Share the Swirl of Friendship with sweetFrog® and Care Bears™', 'How to Find Strong Consumer Discretionary Stocks Slated for Positive Earnings Surprises', '2 Unstoppable Streaming Stocks to Buy Hand Over Fist in 2025, According to Wall Street', "Fubo Gets 'Fairy Tale Outcome': Analyst Says Disney Deal Lowers Risks, Overhang On Stock - FuboTV  ( NYSE:FUBO ) ", 'IPSY Joins Forces With Gabrielle Union to Celebrate Black History Month', "What's Going On With TKO Group Holdings Stock Today? - TKO Group Holdings  ( NYSE:TKO ) ", 'BLACK ENTERPRISE Announces the 40 Under 40 List', "Dana White Wanted Elon Musk, Mark Zuckerberg In Cage Fight, Now He's On Meta's Board Of Directors - Meta Platforms  ( NASDAQ:META ) ", 'Netflix  ( NFLX )  Ascends But Remains Behind Market: Some Facts to Note', 'Investors Look Back at 2024', 'Netflix Brings WWE Monday Night RAW to Streaming Starting January 6 - Netflix  ( NASDAQ:NFLX ) ', "The World's Smallest Laser TV, the JMGO O2S Ultra, Shines at CES 2025", 'reVolver Podcasts--The Preferred Platform with Millions of Downloads--Launches Exclusive App on Samsung Galaxy Store', 'The Best Stocks to Invest $1,000 In Right Now', 'Comcast to launch Universal Ads in bid to win smaller advertisers over from tech', 'How the sports media world is undergoing a seismic shift', 'Melania Trump To Produce Upcoming Amazon Prime Documentary About Her Life and Influence In Trump Political Universe - Amazon.com  ( NASDAQ:AMZN ) , Walt Disney  ( NYSE:DIS ) ', "Wish You'd Bought Palantir in 2024? Buy This Unstoppable ETF Instead", "Consumer Tech News  ( Dec 30-Jan 3 ) : Apple Stops Selling iPhone 14, iPhone 14 Plus, iPhone SE, Trump Seeks A 'Political Resolution' Of TikTok Ban & More - Microsoft  ( NASDAQ:MSFT ) , Apple  ( NASDAQ:AAPL ) ", 'Tesla, Netflix, MicroStrategy - And Shiba Inu Strives To Outpace Dogecoin In Crypto: Bulls And Bears', '"Rule Breaker Investing" Mailbag: Portfolio-Level Thinking', 'Bearish Analyst Regrets Not Upgrading Netflix To Buy In 2023 - Netflix  ( NASDAQ:NFLX ) ', 'MGNI Stock Surges 31% in 3 Months: Is There Room for Further Growth?', 'Company News for Jan 3, 2025', 'Tesla To $351? Here Are 10 Top Analyst Forecasts For Friday - Citigroup  ( NYSE:C ) , Caesars Entertainment  ( NASDAQ:CZR ) ', '5 Dates for Disney Stock Investors to Circle in 2025']]
news_titles = [['News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business', 'News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business', 'T-Mobile Earnings Miss But Subscriber Adds Top AT&T, Verizon', 'Why Netflix Stock Was Soaring This Week', 'WWE boss Vince McMahon accused of sexual assault and trafficking in new lawsuit', 'Bear Call Spread In Netflix Adds $65 In Your Wallet Now', "Good News For 'Squid Game' Fans As Season 2 News Finally Surfaces", "Netflix Is Raising Prices In The Most Cynical Way And It's Only Going To Get Worse", "Here's How Much You Would Have Made Owning Netflix Stock In The Last 20 Years - Netflix  ( NASDAQ:NFLX ) ", 'Tesla: 5 Reasons the Earnings Dip is a Bargain', 'Netflix shares are fully priced, Wall Street analysts say. Here here are six other media stocks to think about.', 'Descendant Filmmaker Honored by Blacks In Government', 'Netflix Shares Surge Post Earnings: ETFs in Focus', 'Netflix  ( NFLX )  to Live Stream WWE RAW Exclusively Starting 2025', "How To Watch The WWII Miniseries 'Masters Of The Air'", 'Best Momentum Stocks to Buy for January 25th', 'Gradle Inc. Achieves Iron Bank Certification and Plans to Play a Key Role in Accelerating Software Delivery Time and Securing the DoD Software Supply Chain', 'Hughesnet Vs. Viasat: Which Is Best For You?', 'Why This 1 Momentum Stock Could Be a Great Addition to Your Portfolio', 'Business | Jan 27th 2024 Edition', 'Apple and Cenovus Energy have been highlighted as Zacks Bull and Bear of the Day', 'Superyacht Designers Bannenberg & Rowell Go BIG With 367-Foot-Long Renaissance', "From Scholar To Coder, Gamer To Seller: Here Are A Few Custom GPTs To Make Everyone's Life Easier - Microsoft  ( NASDAQ:MSFT ) ", 'Bell Fibe TV brings next generation TV and entertainment to Atlantic Canada', 'Bell Fibe TV brings next generation TV and entertainment to Atlantic Canada', 'Digital Player Affirm Holdings Ventures Into The Brick-And-Mortar World', 'Ally Invest Is Growing Its Name Among The Best Online Brokers', 'Futures: Analysts Slash Tesla Views After Earnings Call; Humana Dives', 'Nasdaq, S&P 500 Futures Unfazed By Tesla Disappointment As Focus Shifts To Q4 GDP, Earnings: Analyst Sees Hope In This Bullish Indicator - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) , SPDR S&P 500  ( ARCA:SPY ) ', 'Light shows: after-dark displays in the UK this winter', 'First Busey  ( NASDAQ:BUSE )  Upgraded to Hold at StockNews.com', '3 Reasons Roku Stock Could Crush the Market for the Next Decade', 'IQVIA  ( NYSE:IQV )  Upgraded to Buy by StockNews.com', 'Logitech International  ( NASDAQ:LOGI )  Upgraded to "Strong-Buy" at StockNews.com', 'UFP Technologies  ( NASDAQ:UFPT )  Downgraded to Sell at StockNews.com', 'Trustmark  ( NASDAQ:TRMK )  Downgraded by StockNews.com to Sell', 'Cirrus Logic  ( NASDAQ:CRUS )  Upgraded to "Buy" by StockNews.com', 'Franklin Resources  ( NYSE:BEN )  Upgraded to Buy at StockNews.com', 'Marcus  ( NYSE:MCS )  Upgraded to Buy by StockNews.com', 'EVERTEC  ( NYSE:EVTC )  Downgraded to Hold at StockNews.com', 'General Electric  ( NYSE:GE )  Downgraded by StockNews.com to Hold', 'Stride  ( NYSE:LRN )  Upgraded to "Buy" by StockNews.com', 'MakeMyTrip  ( NASDAQ:MMYT )  Upgraded to Hold by StockNews.com', 'Miller Industries  ( NYSE:MLR )  Downgraded to "Buy" at StockNews.com', 'Boyd Gaming  ( NYSE:BYD )  Upgraded to Buy by StockNews.com', 'International Game Technology  ( NYSE:IGT )  Upgraded to "Strong-Buy" by StockNews.com', 'Altisource Asset Management  ( NYSE:AAMC )  Coverage Initiated by Analysts at StockNews.com', 'CSP  ( NASDAQ:CSPI )  Lowered to "Hold" at StockNews.com', 'Intrepid Potash  ( NYSE:IPI )  Upgraded to "Sell" at StockNews.com', 'Quest Diagnostics  ( NYSE:DGX )  Upgraded to "Buy" by StockNews.com'], ['WEBTOON Entertainment Inc. to Participate in Upcoming Investor Conferences', 'WEBTOON Entertainment Inc. to Participate in Upcoming Investor Conferences - Webtoon Entertainment  ( NASDAQ:WBTN ) ', "Does Netflix's Push Into Live Steaming Justify Buying the Stock Now?", 'Cineverse Releases Holiday Movie Slate to Air on The Dove Channel - Cineverse  ( NASDAQ:CNVS ) ', 'Dinewise, Inc.  ( DWIS )  Appoints Former NFL Star Michael Farr as CEO', 'Asset Entities Signs Agreement with Netflix Star Jas Leverette to Build Digital Dog Training Community - Asset Entities  ( NASDAQ:ASST ) ', 'SPACE FOR HUMANITY EXPRESSES SUPPORT FOR SCIENCE COMMUNICATOR HEADING TO SPACE', "3 Unstoppable Stocks With Competitive Moats That Appear Poised to Become Wall Street's Next Stock-Split Stocks in 2025", "Mohamed El-Erian Warns Against Simplistic Narratives As Trump Plans Aggressive Tariff Strategy: 'The Issue Is Quite Complex' - Applied Mat  ( NASDAQ:AMAT ) , Broadcom  ( NASDAQ:AVGO ) ", 'Unmissable Black Friday Deals on HAPPRUN Projectors - Transform Your Home Theater!', 'RUSSELL PETERS NEW STAND-UP SPECIAL: ACT YOUR AGE Live in Abu Dhabi PREMIERING ON PATREON DECEMBER 5TH, 2024 AT 6PM PST', 'FOX vs. NFLX: Which Stock Should Value Investors Buy Now?', "AMD's HPC Portfolio Powers El Capitan: How Should You Play the Stock?", 'Warner Bros. Discovery Falls 17% YTD: How Should You Play the Stock?', 'The Smurf Experience - A 13,000-Square-Foot Family Attraction - Sets its U.S. Debut for Early 2025', 'Unpacking the Latest Options Trading Trends in Netflix - Netflix  ( NASDAQ:NFLX ) ', 'Atmospheric Scientist Katharine Hayhoe to Discuss Finding Hope in the Face of Climate Change During Public Talk at Dickinson College', 'Qloo® Partners with Special Olympics to Personalize Donor Engagement and Identify New Donors Through Taste AI™', 'Athletic Brewing and Netflix Launch Marine Odyssey IPA', "Bitcoin and Dogecoin Rise While Ethereum Drops; BlackRock's ETF Hits $1B Volume; Walmart Market Share Soars Driven by Six-Figure Income Households - Top Headlines Today While US Slept", 'Netflix Made DVDs Obsolete By Popularizing Streaming. Ben Affleck Thinks AI Will Change The Face Of Hollywood Next - Netflix  ( NASDAQ:NFLX ) , Walt Disney  ( NYSE:DIS ) ', 'TransPerfect Media Named Netflix Preferred Fulfillment Partner of the Year 2024 for APAC Region', '2 Hypergrowth Tech Stocks to Buy in 2024 and Beyond', 'The Future is Here: New Mexico Media Arts Collective Launches First-of-its-Kind Emerging Technology Lecture Series', "Disney's Q4 Earnings Surpass Estimates: How to Play the Stock", "Disney's Stock Is Too Cheap to Pass Up", 'Disney Investors Just Got Fantastic News, but Is the Stock a Buy?', 'Billionaire Jerry Jones Just Explained Why Live Sports Is a No-Brainer for Netflix. Is the Streaming Stock About to Pop Again?', 'Brokers Suggest Investing in Netflix  ( NFLX ) : Read This Before Placing a Bet', "Tesla Is Back In The Future Fund Active ETF's Top 5 Holdings - Tesla  ( NASDAQ:TSLA ) ", 'Netflix Stock Soars Amid Tyson-Paul Boxing Stream Stutters; Analyst Gary Black Says Viewership Signals Q4 Subscriber Growth', 'Warner Bros. Secures 11-Year NBA Deal Extension - Warner Bros. Discovery  ( NASDAQ:WBD ) ', 'Netflix Crash Hits Jake Paul-Mike Tyson Fight: Can Streaming Platform Handle Christmas Day NFL/Beyonce Action? - Netflix  ( NASDAQ:NFLX ) ', 'Netflix Scores Big: Beyoncé Set To Headline NFL Christmas Halftime Show - Netflix  ( NASDAQ:NFLX ) ', 'Disney Shines In All Its Entertainment Glory - Walt Disney  ( NYSE:DIS ) ', "LeBron James' SpringHill to merge with 'The Kardashians' producer Fulwell 73", 'MULTIMEDIA UPDATE -- Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health - Prenetics Global  ( NASDAQ:PRE ) ', 'MULTIMEDIA UPDATE -- Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health', 'Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health', 'Nasdaq-Listed Prenetics and David Beckham Officially Launch IM8 Health - Prenetics Global  ( NASDAQ:PRE ) ', "Warner's Max launches in Asia, taking on Netflix in Hong Kong", 'Mike Tyson Discloses Near-Death Experience Before Jake Paul Fight', 'The Future of Live Sports on Netflix: Lessons Learned From Problems With the Tyson-Paul Event', "Jake Paul Vs. Mike Tyson Boxing Match Sets 'Record-Breaking Night' For Netflix: Streaming Giant Says Nearly 60M Households Tuned In - Netflix  ( NASDAQ:NFLX ) ", 'The Best Stocks to Invest $50,000 in Right Now', "Mike Tyson Vs. Jake Paul: Netflix's Explosive Boxing Event Ends With Paul's Victory - Netflix  ( NASDAQ:NFLX ) ", "The Trade Desk's Valuation Looks Expensive. Can the Stock Outperform in 2025?", 'Time to Buy Home Depot or Disney Stock After Beating Earnings Expectations?', 'Jake Paul, Mike Tyson fight breaks record for biggest boxing gate outside of Las Vegas', "Netflix Streams Mike Tyson Vs. Jake Paul: The Fight Everyone's Talking About - Netflix  ( NASDAQ:NFLX ) "], ['Is Netflix Stock Poised to Grow Again?', 'What Comes Next In The Ukraine War--Plus, Fading The Recent Rally With Bets Against Real Estate - Netflix  ( NASDAQ:NFLX ) ', "Why Netflix Won't Forecast Subscriber Numbers Anymore", 'Video: Formula 1 CEO Stefano Domenicali Talks U.S. Grand Prix, Staying In Austin Long-Term', 'Why Upstart, PayPal, and SoFi Stocks Were Volatile Today', 'Is Netflix Password Sharing An Even Bigger Problem Than People Thought?', 'Are All Streaming Services Losing Money? Netflix Seems To Think So - Netflix  ( NASDAQ:NFLX ) ', "WACO CENTER'S WEARABLE ART GALA RETURNS WITH STEP INTO HARLEM NIGHTS", "India's top multiplex chains bank on festive season to bring cheer back", 'Why Marqeta Stock Jumped Again Today', "India's top multiplex chains bank on festive season to bring cheer back", 'Education Department could flag up to 5 million student loan relief applications for review - what to do if yours is among them', 'FUNRISE, INC. AND PARAMOUNT CONSUMER PRODUCTS PARTNER TO LAUNCH TEENAGE MUTANT NINJA TURTLES TOY CARS FALL 2023 - Paramount Global  ( NASDAQ:PARA ) ', 'CIO Leadership: The Modern Technology Executive - Leading, Inventing and Winning in Unprecedented Times and Uncharted Waters Will Drive the Discussion at the 2022 Greenwich CIO Executive Leadership Summit on November 3', 'AT&T;  ( T )  Q3 2022 Earnings Call Transcript', "Movie theaters want more from Netflix, but the streaming giant isn't ready to budge on its release model", "Box Office: 'Black Panther: Wakanda Forever' Aims For Record $175 Million Debut", 'The Best New Movies And Shows On Netflix Today: October 20, 2022', 'Wall Street rises as upbeat corporate forecasts trump UK political shock', 'Why Is Disney Stock Down 37% This Year?', 'Higher Rates vs. Q3 Earnings - Blackstone  ( NYSE:BX ) , FTX Token  ( FTT/USD ) ', 'Business', 'Stock Market News for Oct 20, 2022', "I've Been a Netflix Shareholder for 20 Years", "US Stocks: S&P 500 to open lower as UK's Truss resigns as PM | The Financial Express", 'Netflix Stock Surges on Earnings Beat and Subscriber Growth: 6 Metrics You Should See', 'Better Buy: Coca-Cola vs. Disney', 'Is Netflix Stock a Buy Now?', 'Haas F1 inks MoneyGram to 3-year title sponsorship deal', 'Futures Rise; Tesla Skids On Sales, Musk', "This Analyst With 81% Accuracy Rate Upgrades Netflix; Here Are 4 Other Stock Picks For October 19 From 's Most Accurate Analysts - Carvana  ( NYSE:CVNA ) , Equitable Holdings  ( NYSE:EQH ) ", "Warner Bros Releases Ethereum-Compatible 'Lord of the Rings' NFTs", 'The Zacks Analyst Blog Highlights Netflix, Intuitive Surgical, Lockheed Martin and United Airlines', 'Nasdaq futures fall as Tesla warns on 2022 deliveries', '3 Explosive Stocks to Buy Right Now', 'Weekender: 25 things to do in Charlotte this weekend including pumpkin carving and chili tasting', 'Intuitive Surgical, Netflix, ASML And Other Big Gainers From Wednesday - Amprius Technologies  ( NYSE:AMPX ) , ASML Holding  ( NASDAQ:ASML ) ', 'Zacks Investment Ideas feature highlights: Netflix, Microsoft and Disney', 'Investor Fear Eases Slightly Ahead Of Earnings Results - American Airlines Group  ( NASDAQ:AAL ) , Amazon.com  ( NASDAQ:AMZN ) ', 'Talenthouse to partner with and invest up to $7 million in cash and shares in US-based digital SME banking provider, Nerve.', 'Talenthouse to partner with and invest up to $7 million in cash and shares in US-based digital SME banking provider, Nerve.', 'Global stocks lower amid British political turmoil', 'Asian shares fall after weak earnings pull Wall St lower', 'Asian Markets Drop And Dollar Rises As Inflation, Rate Fears Return', 'Asian Markets Drop And Dollar Rises As Inflation, Rate Fears Return', 'Equities close lower as rise in yields overshadows earnings', 'While Netflix Pushes Quantity, Quality Matters More To Most Viewers, UTA Data Suggests', "Buy shares of Netflix on the market's next pullback, Jim Cramer says", 'Why Roku Stock Popped Then Dropped on Wednesday', "Controversy Over 'The Crown' Grows: Judi Dench Says Netflix Should Add Disclaimers To Series"], ['NFL Games Set Thanksgiving Day Records: Why This Is Good And Bad News For Streaming Giant Netflix - Netflix  ( NASDAQ:NFLX ) ', 'Trakt Partners with Younify to launch its Streaming Scrobbler', "Aaron Rodgers Opens Up About Psychedelic Healing In New Netflix Docuseries, 'Enigma'", "Netflix Stock Reaches New All-Time High This Week: What's Driving The Surge? - Netflix  ( NASDAQ:NFLX ) ", 'Is Netflix  ( NFLX )  a Buy as Wall Street Analysts Look Optimistic?', "MasterClass, Women's Health Advocate Halle Berry and Leading Health Experts Come Together for Stigma-Breaking Class on Menopause", 'Toyota Spreads Festive Cheer This Holiday Season - Toyota Motor  ( NYSE:TM ) ', '1 No-Brainer Vanguard Sector ETF to Buy Right Now for Less Than $500', 'The Top 16 Stocks to Buy Now in December  ( 2024 ) ', '3 Breakout Growth Stocks You Can Buy and Hold for the Next Decade', 'Netflix  ( NFLX )  Surpasses Market Returns: Some Facts Worth Knowing', 'Roku Shares Are On The Rise Today: What You Need To Know - Roku  ( NASDAQ:ROKU ) ', 'The Trade Desk Just Made a Big Move That Has Investors Scrambling to Buy Roku Stock', "Peacock Network to Premiere Visit Myrtle Beach's TV Series 'Traveling the Spectrum' on Dec. 12", 'Netflix Tops 10 Million Subscribers In Japan - Netflix  ( NASDAQ:NFLX ) ', '5 Stocks Driving Nasdaq ETF to New Highs in 2024', 'Traditional Pay-TV Subscriptions to Dip Below 35% in 2025 as Streaming Becomes the Norm - AlixPartners Annual Media Predictions', 'Top 3 Tech And Telecom Stocks That May Implode This Quarter - Meta Platforms  ( NASDAQ:META ) , Walt Disney  ( NYSE:DIS ) ', 'Is Bluesky the Killer App That Will Kickstart the Web3 Revolution in 2025?', 'Up 137%, Should This Red-Hot Artificial Intelligence Cryptocurrency Be in Your Portfolio for 2025?', 'Should You Buy Amazon Stock Before Dec. 19?', 'Consumer Electronics Manufacturing Market to Surge Valuation Set to Hit US$ 4,267.27 Billion by 2034 at 9.5% CAGR | Fact.MR Report', "'Squid Game' Season 2 Includes New Games, Contestant With A Crypto Background, Revenge Factor, Political Overtones - Netflix  ( NASDAQ:NFLX ) ", 'Verizon turns up the heat on savings with Holiday Deal Days', 'Rule Breaker Investing: The 2024 Gratitude Episode', 'Company News for Dec 3, 2024', 'SCHOLASTIC UNLEASHES DOG MAN GLOBAL CAMPAIGN CELEBRATING READING AND HEROES AROUND THE WORLD - Scholastic  ( NASDAQ:SCHL ) ', 'TWLO Enhances Its Segment Platform: What Should Investors Do?', "Domino's® and Netflix Partner to Bring Emergency Pizza to the Squid Game Universe - Netflix  ( NASDAQ:NFLX ) , Domino's Pizza  ( NYSE:DPZ ) ", "This Vanguard ETF Is Up 32% in 2024. Here's Why It's a Simple Way to Invest in Google Parent Alphabet, Meta Platforms, Netflix, and Disney.", 'From Shopping to Advice From Warren Buffett', "How Dick's Sporting Goods Is Succeeding", "Macy's Multimillion-Dollar Error", 'Trump\'s Return to the White House Could Spark Unprecedented Investment Opportunity: Experts Eye Starlink\'s Potential "Super-IPO"', 'Netflix To Rally More Than 7%? Here Are 10 Top Analyst Forecasts For Monday - Applied Therapeutics  ( NASDAQ:APLT ) , Affirm Holdings  ( NASDAQ:AFRM ) ', 'Costco Sales Topped $250 Billion in Fiscal 2024. But Less Than 2% of Those Sales Are Key to Understanding the Entire Business.', 'Could Buying Nu Stock Today Set You Up for Life?', 'If I Could Only Buy and Hold a Single Stock, This Would Be It', 'Roku  ( ROKU )  Up 5.7% Since Last Earnings Report: Can It Continue?', 'Is the iShares Expanded Tech Sector ETF a Millionaire Maker?', "Is fuboTV Stock a Buy Following Netflix's Live Sporting Event Debacle?"], ['Morning Bid: Five alive: US yield curve nears historic level', 'Stock Market Today: Powell Sends Stocks On Roller-Coaster Ride', "US stocks fall as the 10-year US Treasury yield approaches 5% and Powell says inflation 'still too high'", "Netflix price increase: Here's how much the major streaming services are set to cost", 'Dow Jones Dives Amid Powell Pledge; Tesla Sinks After Elon Musk Warning', 'Netflix Thrived Despite Hollywood Tensions - Netflix  ( NASDAQ:NFLX ) ', "As Netflix's stock heads for best one-day in almost three years, investors are snapping up its bonds", 'Netflix Hikes Prices Again As Subscriptions Climb', 'A travel influencer shared a plethora of tips on how to best sleep in airports, including chaining luggage to your body with earbud wires', 'Netflix Rallies Off Q3 Earnings: 8 Analysts On Future Price Increases, Ad-Supported Plans And More - Netflix  ( NASDAQ:NFLX ) ', 'NFLX: 3 Internet Stocks to Keep Watching for Future Gains', 'Dow Jumps 150 Points; Netflix Earnings Top Views - SeaStar Medical Holding  ( NASDAQ:ICU ) , Lindsay  ( NYSE:LNN ) ', 'Wall Street Cheers Netflix Price Increases As Sign Of Strength', "Five In The Wallet: 10-Year Yield Flirts With 5% For First Time Since 2007 As Fed's Powell Speech Awaited - CSX  ( NASDAQ:CSX ) , American Airlines Group  ( NASDAQ:AAL ) ", 'Netflix  ( NFLX )  Q3 Earnings Beat, Revenues Rise Y/Y on User Gain', 'Why Netflix Stock Jumped Back Over $400 Per Share Today', 'Netflix Shares Surge 15% After Subscriber Growth-Largest Single-Day Gain Since 2021', 'Tesla & Netflix Earnings: A Closer Look', 'Why Streaming Service Roku Inc Stock Is Up Today - Netflix  ( NASDAQ:NFLX ) , Roku  ( NASDAQ:ROKU ) ', 'Stock Market Erases Gains And Here Comes Powell; Tesla Stock Has This Problem', 'Market Clubhouse Morning Memo - October 19th, 2023  ( Trade Strategy For SPY, QQQ, AAPL, MSFT, NVDA, TSLA, And NFLX )  - Invesco QQQ Trust, Series 1  ( NASDAQ:QQQ ) ', 'Stocks Mixed After Jobless Claims', 'Netflix stock surges 16% after Wall Street buys into ad-driven subscriber growth', "ETFs to Buy on Netflix's Blowout Q3 Earnings", 'Tesla, Netflix Earnings: Everything You Need to Know', 'AT&T, Union Pacific, Las Vegas Sands And Other Big Stocks Moving Higher On Thursday - American Airlines Group  ( NASDAQ:AAL ) , Badger Meter  ( NYSE:BMI ) ', "US stocks rise as key Treasury yield nears 5% ahead of Powell's speech", "Hollywood's actors strike is nearing its 100th day. Why hasn't a deal been reached and what's next?", 'Netflix Stock: Bear vs. Bull', "Netflix Stock Fast Forwards As Subscribers Surge: What's Going On Thursday? - Netflix  ( NASDAQ:NFLX ) ", 'The top 10 things to watch in the stock market Thursday', 'Netflix Realigns Pricing Strategy In Australia, Ends Basic Plan For New Members - Netflix  ( NASDAQ:NFLX ) ', 'Netflix Stock Jumps 13% on Earnings Beat and Strong Subscriber Growth', "Martin Scorsese's 'Killers of the Flower Moon' wants to be epic but just feels long", 'Where to look to find enlightenment', 'Business | Oct 21st 2023 Edition', 'The daily business briefing: October 19, 2023', 'Best Buy To Rally More Than 21%? Here Are 10 Top Analyst Forecasts For Thursday - First Solar  ( NASDAQ:FSLR ) , Best Buy Co  ( NYSE:BBY ) ', 'Why Netflix Shares Are Trading Higher By Around 14%; Here Are 20 Stocks Moving Premarket - Biocept  ( NASDAQ:BIOC ) , Atreca  ( NASDAQ:BCEL ) ', "Netflix's Ads Business Is Taking Off", "Futures inch up ahead of Powell's comments; Tesla drops, Netflix surges", 'Netflix adds $28 billion in market value after password crackdown drives subscriber spike', "Netflix's total valuation is set to balloon by over $20 billion after the streaming giant said subscriber numbers had soared", 'Netflix adds $28 billion in market value after password crackdown drives subscriber spike', "S&P 500, Nasdaq Set For Mixed Open After Tesla, Netflix Earnings: What's Moving Stock Futures? - Alaska Air Gr  ( NYSE:ALK ) , Netflix  ( NASDAQ:NFLX ) , VMware  ( NYSE:VMW ) , Broadcom  ( NASDAQ:AVGO ) ", 'Stanley Druckenmiller said central banks, not earnings, move markets. Today is the day to pay attention.', '5 things to know for Oct. 19: Israel, House speaker, Covid-19, Pollution, UFOs', 'Futures: Yields Eye 5%; Tesla Falls On These Elon Musk Warnings', 'Morning Bid: To 5% and beyond, bond yields soar', "Frasier review: a 'gentle' reboot of the beloved 1990s sitcom"]]
news_titles = list(itertools.chain.from_iterable(news_titles))
news_titles_appended = ""
for news_title in news_titles:
    news_titles_appended = news_titles_appended + "," + "\"" + news_title + "\""

print(news_titles_appended)

,"News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business","News industry off to brutal 2024 start as mass layoffs devastate publishers, raising questions about the future of journalism | Business","T-Mobile Earnings Miss But Subscriber Adds Top AT&T, Verizon","Why Netflix Stock Was Soaring This Week","WWE boss Vince McMahon accused of sexual assault and trafficking in new lawsuit","Bear Call Spread In Netflix Adds $65 In Your Wallet Now","Good News For 'Squid Game' Fans As Season 2 News Finally Surfaces","Netflix Is Raising Prices In The Most Cynical Way And It's Only Going To Get Worse","Here's How Much You Would Have Made Owning Netflix Stock In The Last 20 Years - Netflix  ( NASDAQ:NFLX ) ","Tesla: 5 Reasons the Earnings Dip is a Bargain","Netflix shares are fully priced, Wall Street analysts say. Here here are six other media stocks to think about.","Descendant Filmmaker Honored by Blacks In Governmen